<a href="https://colab.research.google.com/github/ATalaveraDev/social-network-extraction-info/blob/main/ERS_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Dependencias

In [ ]:
pip install tweepy

###Obtener los Estados de Estados Unidos

En esta parte he hecho scrapping de una web del google developers portal que contiene el codigo de cada estado, su nombre y sus coordenadas. El motivo de esto es porque de cada Tweet voy a recoger el valor del atributo "location" el cual presenta el formato: "nombre_estado, CODIGO_ESTADO"

In [34]:
import requests
from bs4 import BeautifulSoup

URL = "https://developers.google.com/public-data/docs/canonical/states_csv"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

states = []
table = soup.select_one('table')
for row in table.select('tr'):
  tds = row.find_all('td')
  if (len(tds) > 0):
    cells = [d.text for d in tds]
    states.append({ 'state': cells[0], 'name': cells[3], 'latitude': cells[1], 'longitude': cells[2] })

# DEBUGGING del resultado
#states

OAuth authentication

In [35]:
import tweepy

auth = tweepy.OAuthHandler('GJdVKUjoEl31WLHlgIRYxVmYr', 'HVaOvQWoHIkiHZh2yPNb0YMxwR6waHagtoR4hcKURo6nhvGxwL')
auth.set_access_token('1254452545583529991-g0sI2XhrtijaQ5zA3NNPRwCfuSKrCr', '4GLEprsRqsKlHPelKC4YirECbHTQYsJJGYiy0JN2OuRiH')

api = tweepy.API(auth)

###Tweets Selection

En esta sección se obtienen los Tweets a través de la funcionalidad de streaming de la API. Por defecto el tiempo de streaming es de 1 minuto pero se puede configurar, para ello basta con editar la linea "time.sleep(60)"

In [36]:
from tweepy.streaming import StreamListener
from tweepy import Stream
import json
import time

class StdOutListener(StreamListener):
    def on_data(self, data):
      tweet = json.loads(data)
      tweets.append({ 'text': tweet['text'], 'place': tweet['place']['full_name'] })
      return True

    def on_error(self, status):
        print(status)

tweets = []
l = StdOutListener()
stream = Stream(auth, l)
stream.filter(languages=['en'], locations=[-171.791110603, 18.91619, -66.96466, 71.3577635769], is_async=True)
time.sleep(30)
stream.disconnect()


###Análisis de Sentimiento

Es necesario importar al proyecto el archivo AFINN-111.txt. Tras ello he configurado el programa para que haga un parser del documento y así crear un dictionary de sentimientos que será utilizado posteriormente para asignar a cada Tweet un "índice de felicidad"

In [37]:
feelings = open('AFINN-111.txt', 'r')

def initFeelingsDictionary():
  dict = {}
  for feeling in feelings:
    feelingSplit = feeling.split('\\n')
    feel = feelingSplit[0].replace('\n', '').split('\t')
    dict[feel[0]] = feel[1]
  return dict

affinn = initFeelingsDictionary()

# Para evitar problemas con los streamings recogí algunos tweets y los utilice como datos. De esta forma no habia necesidad de utilizar 
# los streams en todas las fases del desarrollo
#testData = [{'text': '@barbiebayxx @NaijaEverything We from Africa it ain’t née plus I like younger girls 🌚🌚 she looks sweet', 'place': 'Georgia, USA'}, {'text': '@lonliestwolf Gooood....goooood https://t.co/QdQKaWl8xK', 'place': 'Columbus, OH'}, {'text': 'it’d be a lie to say that shit didn’t hurt my feelings.', 'place': 'Dallas, TX'}, {'text': 'I can’t complain...🤷🏾\u200d♂️... it was all 4 the taking 🙇🏾\u200d♂️', 'place': 'Oakland, CA'}, {'text': 'Flat Rock / Coon Creek: Running https://t.co/XnDGONy515', 'place': 'Alabama, USA'}, {'text': 'Don’t let No one get comfortable Disrespecting You!!!', 'place': 'Scio, MI'}, {'text': '@Thomas1774Paine Just like he was a Russian puppet right Eric? \nHow’s that Russian Collusion fairytale going? Have… https://t.co/38iy7V6MgC', 'place': 'Russellton, PA'}, {'text': '@Oluomachi_u I believe in you!😭', 'place': 'Chicago, IL'}, {'text': '@iTS_MeMadiO You work at the 🏥', 'place': 'Dallas, TX'}]

countryHappinness = {}
#for tweet in testData:
for tweet in tweets:
  if ',' in tweet['place']:
    code = tweet['place'].split(', ')[1]
    for value in states:
      if value['state'] == code:
        happinessIndex = 0
        for word in tweet['text'].split():
          if word.lower() in affinn:
            happinessIndex += int(affinn[word.lower()])
        if value['state'] in countryHappinness:
          totalHappiness = countryHappinness[value['state']]['happiness'] + happinessIndex
        else:
          totalHappiness = happinessIndex
        countryHappinness.update({ value['state']: { 'happiness': totalHappiness, 'latitude': value['latitude'], 'longitude': value['longitude'], 'name': value['name'] } })
print(countryHappinness)

{'TX': {'happiness': 5, 'latitude': '31.968599', 'longitude': '-99.901813', 'name': 'Texas'}, 'OH': {'happiness': 2, 'latitude': '40.417287', 'longitude': '-82.907123', 'name': 'Ohio'}, 'CT': {'happiness': 11, 'latitude': '41.603221', 'longitude': '-73.087749', 'name': 'Connecticut'}, 'NY': {'happiness': 6, 'latitude': '43.299428', 'longitude': '-74.217933', 'name': 'New York'}, 'GA': {'happiness': -4, 'latitude': '32.157435', 'longitude': '-82.907123', 'name': 'Georgia'}, 'CA': {'happiness': 8, 'latitude': '36.778261', 'longitude': '-119.417932', 'name': 'California'}, 'IL': {'happiness': 1, 'latitude': '40.633125', 'longitude': '-89.398528', 'name': 'Illinois'}, 'NC': {'happiness': 6, 'latitude': '35.759573', 'longitude': '-79.0193', 'name': 'North Carolina'}, 'MI': {'happiness': 1, 'latitude': '44.314844', 'longitude': '-85.602364', 'name': 'Michigan'}, 'NV': {'happiness': 6, 'latitude': '38.80261', 'longitude': '-116.419389', 'name': 'Nevada'}, 'FL': {'happiness': -8, 'latitude': '

## Map

In [7]:
!pip install rasterio
!pip install earthpy

     |████████████████████████████████| 19.1MB 1.3MB/s 
     |████████████████████████████████| 1.4MB 3.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0MB 42.6MB/s 
     |████████████████████████████████| 6.5MB 51.1MB/s 
     |████████████████████████████████| 14.8MB 336kB/s 
  Created wheel for earthpy: filename=earthpy-0.9.2-cp37-none-any.whl size=1365089 sha256=fd8db22055701c0c839057505e5f59407398e1fe58151e1d7304edaa9e1e8d1f
  Stored in directory: /root/.cache/pip/wheels/27/f9/a5/026bb95ef7b12f9343b842da6897b4001c5b902550095cbbbe
Successfully built earthpy


In [8]:
import os 
import folium
from folium import plugins
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy as et

In [38]:
# Crear el mapa con centro en USA
m = folium.Map(location=[37.09024,	-95.712891], zoom_start=4)

# Añadir Markers al mapa
for country in countryHappinness:
  color = 'black'
  happinessLevel = countryHappinness[country]['happiness']
  if happinessLevel < -2:
    color = 'red'
  elif happinessLevel >= -2 and happinessLevel <= 2:
      color = '#BDB76B'
  elif happinessLevel > 2:
      color = 'green'
  folium.Marker(
    location=[float(countryHappinness[country]['latitude']), float(countryHappinness[country]['longitude'])],
    icon=folium.DivIcon(html=f"""<div style="color: {color}; font-size: 14px">{countryHappinness[country]['happiness']}</div>""")
  ).add_to(m)

# Mostrar mapa
m

# CONCLUSIÓN: 5 ESTADOS MÁS FELICES



In [40]:
import operator

result = []
for s in sorted(countryHappinness.items(), key=lambda k_v: k_v[1]['happiness']):
  result.append(s)
result[len(result) - 5:len(result)]

[('NJ',
  {'happiness': 6,
   'latitude': '40.058324',
   'longitude': '-74.405661',
   'name': 'New Jersey'}),
 ('CA',
  {'happiness': 8,
   'latitude': '36.778261',
   'longitude': '-119.417932',
   'name': 'California'}),
 ('PA',
  {'happiness': 10,
   'latitude': '41.203322',
   'longitude': '-77.194525',
   'name': 'Pennsylvania'}),
 ('CT',
  {'happiness': 11,
   'latitude': '41.603221',
   'longitude': '-73.087749',
   'name': 'Connecticut'}),
 ('MN',
  {'happiness': 12,
   'latitude': '46.729553',
   'longitude': '-94.6859',
   'name': 'Minnesota'})]